In [1]:
import numpy as np 
import pandas as pd 
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig
import keras 
from keras.preprocessing.sequence import pad_sequences #2.2.4
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

Using TensorFlow backend.


In [2]:
import sklearn 
sklearn.__version__

'0.23.1'

In [3]:
import tensorflow 

In [4]:
tensorflow.__version__

'2.1.0'

In [5]:
torch.__version__

'1.5.1'

In [6]:
transformers.__version__


'2.11.0'

In [13]:
tokenizer_chinese = BertTokenizer.from_pretrained("bert-base-chinese", do_lower_case=False)
tag_values = ['O',
 'B_person_name',
 'M_person_name',
 'E_person_name',
 'PAD']
tag2idx = { 'O': 0,
           'B_person_name': 1,
           'M_person_name': 2,
           'E_person_name': 3,
           'PAD': 4}
# Model1 
PATH = 'step1_1output_bertmode_step1_ner.pth'
model = torch.load(PATH)
model.eval()
# Model class must be defined somewhere
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        output = self.drop(pooled_output)
        return self.out(output)
PATH = 'step3_output_bert_senti.pth'#'bertmode_step2_senti_model1_目前成效最好.pth'

model2 = torch.load(PATH)
model2.eval()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def split_content(x):
    
    if len(x)<=500:
        return [x]
    elif (len(x)>=500) and(len(x)<1000):
        return [ x[:500+3],x[500-6:] ]
    
    elif (len(x)>=1000) and (len(x)<1500):
        return [ x[:500+3],x[500-3:1000+3] ,x[1000-3:]]
    
    elif (len(x)>=1500) and (len(x)<2000):
        return [ x[:500+3],x[500-3:1000+3] ,x[1000-3:1500+3], x[1500-3:]]
    
    else:
        return [x[:500+3],x[500-3:1000+3] ,x[1000-3:1500+3], x[1500-3:2000-3] , x[2000-3:2000-3+500] ]

def find_all_indexes(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1    

In [17]:
def predict(txt):
    """ Predict your model result
    @param article (str): a news article
    @returns prediction (list): a list of name
    """

    ####### PUT YOUR MODEL INFERENCING CODE HERE #######
    test_sentence = split_content(txt)
    row_name = []
    # model1 predict 產生文章中的 name list
    for sentence in test_sentence:
        # bert預測
        tokenized_sentence = tokenizer_chinese.encode(sentence)
        input_ids = torch.tensor([tokenized_sentence]).cuda()

        with torch.no_grad():
            output = model(input_ids)
        label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
        tokens = tokenizer_chinese.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, label_indices[0]):
            if token.startswith("##"):
                new_tokens[-1] = new_tokens[-1] + token[2:]
            else:
                new_labels.append(tag_values[label_idx])#ex:['O','O','O','O',...]
                new_tokens.append(token)# ex:['[CLS]', '益', '公', '司', '債', '或', '新',...]
        texto=''
        for i in range(len(new_labels)):
            if new_labels[i] !='O':
                texto+= new_tokens[i]
            else:
                texto+='O' #'OOO張堯勇OOOOOOOOOOOOO'
        for i in texto.split('O'):
            if len(i)>1:#['張堯勇', '張堯勇']，單一個字或者空白的會被削去
                 row_name.append(i)
    uniq_name = list(set(row_name)) #['鄭心芸', '巴菲特', '詹姆斯·西蒙斯', '堯勇', '索羅斯', '張堯勇']
    
    
    article_truncate = [] # 依據名字擷取的文章
    article_tags = [] # 該人是否是黑名單
    article_tags_name = [] # 該人是誰
    article_len = len(txt)#文章長度
    article_content = txt#文章內文
    all_name_list = uniq_name# 找出共有幾個人
    for j in all_name_list: # 第i篇文章的第j個人
        name_index = find_all_indexes(article_content, j )#出現在文章的index
        for k in name_index:
            article_tags_name.append(j)
            txt2 = txt[  np.clip(k-min(100 , int(article_len*0.2))
                                             , 0 ,999999
                                                  )  : 
                                  np.clip(k-min(50 , int(article_len*0.2))
                                             , 0 , 99999
                                         )+250
                                        ].replace( j , '李家賢') 
            article_truncate.append(txt2)
    df = pd.DataFrame(data={'article_truncate':article_truncate,
                  'article_tags_name':article_tags_name})
    # model2 predict
    prediction_list = []
    for i in range(len(df)):
        review_text = df.iloc[i].article_truncate
        encoded_review = tokenizer_chinese.encode_plus(
        review_text,
        max_length=100,#MAX_LEN,
        add_special_tokens=True,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt')
        input_ids = encoded_review['input_ids'].to(device)
        attention_mask = encoded_review['attention_mask'].to(device)
        output = model2(input_ids, attention_mask)
        _, prediction = torch.max(output, dim=1)
        prediction_list.append(prediction.tolist()[0])
        #print(prediction.tolist()[0])
    df['prediction'] = prediction_list   
    if len(df)==0:
        prediction = []
    else:

        df_group = df.groupby('article_tags_name').mean().apply(lambda x:np.where(x>=0.35,1,0)) # 0.35 是 threshold能調整
        prediction = df_group[df_group.prediction==1].index.tolist()
    ####################################################
    #prediction = _check_datatype_to_list(prediction)
    return list(set(prediction)) ,uniq_name

In [18]:
txt = '''台北地檢署偵辦國安特勤私菸案，歷經1個月多馬不停蹄、抽絲剝繭調查，今偵查終結，將於10點30分開記者會說明。因涉案人數眾多，定調以「嚴辦首謀」、「從犯從輕」為原則，這波偵結對象是以「在押、首謀」為主，擬以貪汙治罪條例第4條第1項第4款「以公用運輸工具裝運違禁物品或漏稅物罪」、第6條第1項第4款「圖利罪」起訴國安局及華航人員。專案小組認定國安局少校吳宗憲、張恒嘉與華航空品處前副總邱彰信、協理于堯、代理組長黃川禎等人涉犯貪汙治罪條例，另買菸大戶上士駕駛劉尊彰、民間人士李宗原等人涉違反稅捐稽徵法，依法起訴。其餘單純團購人員則另簽分偵辦。7月22日，總統蔡英文結束12天「自由民主永續之旅」返台，調查局以行政檢查方式，攔查國安局官員隨團走私9800條菸，金額高達645萬元，創桃園機場夾帶走私菸最大數量紀錄。全案爆發後，北檢立刻組成專案小組。檢方先親赴吳、張位於總統官邸附近侍衛室的宿舍、辦公室，調取電磁紀錄、筆記本、存摺等。鞏固物證後，再大規模約談訂購名單上的買家，查明購菸目的屬自用、代購、轉售。檢調清查華航是誰下令開放免稅菸品無限訂購，幫助特勤人員逃漏稅，8月1日約談華航前資深副總羅雅美、空品處前副總邱彰信等人。複訊後，檢方命邱以70萬元交保、羅雅美則訊後請回、華航空品處協理于堯坦承聽從邱指令辦事，50萬元交保。為釐清華航「機邊交貨」的緣起和運作模式，專案小組向上溯源，傳喚空品處代理組長黃川禎、空品處前後任經理沈珉等華航人員，另傳喚透過總統府侍衛室及國安局訂私菸的民眾共10餘人，訊後全數請回。最後階段，檢調陸續清查訂購數百條的「大戶」，約談華航年輕空服員白梓佑、總統專機御廚田佳宜、華航空品處資深員工陳穎彥、徐世立等人，訊後依違反稅捐稽徵法，各以5萬元至20萬元交保。台北地檢署。圖／本報資料照片facebook'''    

In [19]:
prediction, uniq_name = predict(txt)

In [22]:
prediction

['徐世立',
 '沈珉',
 '黃川禎',
 '雅美',
 '邱彰信',
 '劉尊彰',
 '白梓佑',
 '李宗原',
 '于堯',
 '陳穎彥',
 '張恒嘉',
 '蔡英文',
 '羅雅美',
 '吳宗憲',
 '田佳宜']

In [67]:
s1 = pd.Series(prediction)
s1[s1.str.contains('雅美', regex=False)]

0     雅美
7    羅雅美
dtype: object